# VeADK 教程

## 1. 介绍

VeADK 是由火山引擎推出的一套面向智能体开发的全流程框架，旨在为开发者提供面向智能体构建、云端部署、评测与优化的全流程开发。相较于现有的智能体开发框架，VeADK 具备与火山引擎产品体系深度融合的优势，帮助开发者更高效地构建企业级 AI 智能体应用。

## 2. 配置

### 安装 VeADK

In [ ]:
!pip install veadk-python --quiet

> 如果你在本地使用的终端是`zsh`，当你进行`pip install`时，**依赖包名称应当被双引号包裹起来**，否则会解析错误，例如：
```bash
pip install "veadk-python"
pip install "agent-pilot-sdk>=0.0.9"
```

### 环境变量

In [ ]:
import os

os.environ["MODEL_AGENT_API_KEY"] = "" # <-- 设置火山方舟的API KEY来访问模型

os.environ["LOGGING_LEVEL"] = "ERROR" # <-- 调整日志等级

## 3. Agent 构建

### 快速开始

In [ ]:
from veadk import Agent

agent = Agent() # 设置环境变量后，你无需传入任何参数；VeADK 会自动读取环境变量。默认使用模型为豆包1.6

res = await agent.run("Hello, I am VeADK.")

print(res)

Hello VeADK! It's a pleasure to connect with you. As an AI agent developed by the VeADK team, I'm here to assist with whatever you need—whether it's data science tasks, documentation writing, coding projects, or problem-solving using tools. Just let me know how I can help today! 😊


### 使用 Runner 来运行 Agent

生产环境下推荐使用 Runner 来运行 Agent，以实现多租户隔离。

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk-playground-app"
user_id = "veadk-playground-user"
session_id = "veadk-playground-session"

agent = Agent()
short_term_memory = ShortTermMemory() # 短期记忆代表的是用户单次会话内的对话记录

runner = Runner(
    agent=agent,
    short_term_memory=short_term_memory,
    app_name=app_name,
    user_id=user_id
    )

response = await runner.run(messages="Hello, I am VeADK!", session_id=session_id)

print(response)

Hello VeADK! It's wonderful to connect with you—since I'm an AI agent developed by your team, I’m here to support with whatever you need. Whether it’s data science tasks (like analysis or fact-checking), documentation (articles, reports), coding (web/app development, debugging), or leveraging tools for specific projects, just let me know how I can assist! 😊


### 使用 Tools

使用`tools`字段来挂载工具：

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory
from veadk.tools.demo_tools import get_city_weather

app_name = "veadk-playground-app"
user_id = "veadk-playground-user"
session_id = "veadk-playground-session"

agent = Agent(tools=[get_city_weather])
short_term_memory = ShortTermMemory()

runner = Runner(
    agent=agent,
    short_term_memory=short_term_memory,
    app_name=app_name,
    user_id=user_id
    )

response = await runner.run(messages="北京的天气怎么样？", session_id=session_id)

print(response)

北京现在天气晴朗，气温25°C。


你还可以体验火山引擎提供的网页搜索能力，例如使用 web_search 和 vesearch 工具：

In [ ]:
import os

# 设置火山引擎 AK 和 SK 来使用 web_search 工具
os.environ["VOLCENGINE_ACCESS_KEY"] = ""
os.environ["VOLCENGINE_SECRET_KEY"] = ""

# 设置 vesearch 工具的 ENDPOINT 和 API KEY
os.environ["TOOL_VESEARCH_ENDPOINT"] = ""
os.environ["TOOL_VESEARCH_API_KEY"] = ""

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory
from veadk.tools.builtin_tools.vesearch import vesearch
from veadk.tools.builtin_tools.web_search import web_search

app_name = "veadk-playground-app"
user_id = "veadk-playground-user"
session_id = "veadk-playground-session"

agent = Agent(tools=[vesearch]) # 或者替换为 web_search
short_term_memory = ShortTermMemory()

runner = Runner(
    agent=agent,
    short_term_memory=short_term_memory,
    app_name=app_name,
    user_id=user_id
    )

response = await runner.run(messages="今天的新闻", session_id=session_id)

print(response)

<|FCResponseBegin|>根据搜索结果，2025年4月12日的国内外重点新闻如下：

### **国内新闻**  
#### **社会与安全**  
- **三亚游客溺亡事件**：当日17时25分，三亚崖州区某景区一名游客擅自进入非游览海岸礁石区并跃入海中，景区监控发现后联动公安、医疗展开搜救。17时51分游客被救上岸，但经抢救无效溺水身亡，排除他杀。家属已于4月14日抵达，景区及警方已通报调查结果。  
- **安徽六安风灾致死**：11时20分，六安市金安区因大风天气导致路边树木倒伏，砸中一名55岁电动自行车驾驶员申某某，送医后不治身亡。当地街道办已对家属进行慰问。  

#### **文体与民生**  
- **李荣浩演唱会延期**：受河北大风橙色预警（西北风6-8级，阵风10-12级）影响，原定于当日举办的“李荣浩黑马世界巡回演唱会石家庄站”紧急延期。主办方强调“安全第一”，退票及改期方案将另行通知。  

#### **经济与企业**  
- **年报披露动态**：科创信息（300730.SZ）、汉宇集团（300403.SZ）同日发布2024年年报，首开股份（600376.SH）也计划于4月内披露。市场关注其营收、利润等核心财务指标。  
- **政府采购公告**：某单位公示2025年审计服务采购意向，拟委托第三方开展“2024-2025年度内部控制评价、风险评估及财务收支审计”，预算未公开，预计4月启动。  

### **国际新闻**  
目前公开渠道暂未检索到2025年4月12日的具体国际新闻事件。结合全球动态趋势，当日可能涉及以下领域常规进展：  
- **气候变化合作**：多国在联合国框架下推进碳减排细则谈判，聚焦发展中国家技术转移支持机制。  
- **科技伦理讨论**：美国某科技巨头宣布新一代AI模型商用计划，引发行业对数据隐私与算法监管的争议。  
- **欧洲货币政策**：欧洲央行维持基准利率不变，强调需持续抑制通胀，同时关注能源价格波动对经济复苏的影响。  

（注：国际新闻部分基于常规领域动态梳理，具体事件请以路透社、BBC等权威媒体后续报道为准。）  

如需进一步了解某类新闻细节，可提供关键词获取定向信息。


### 短期记忆

你可以初始化一个`Local`模式的短期记忆，它只会存在与内存中。其仅在一次调用中会有记忆。

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk-playground-app"
user_id = "veadk-playground-user"
session_id = "veadk-playground-session"

agent = Agent()
short_term_memory = ShortTermMemory(backend="local") # 指定 local 后端，或直接 ShortTermMemory()

runner = Runner(
    agent=agent,
    short_term_memory=short_term_memory,
    app_name=app_name,
    user_id=user_id
    )

response = await runner.run(messages=["我叫VeADK", "你还记得我叫什么吗？"], session_id=session_id)

print(response)

当然记得！你叫VeADK，很高兴再次和你交流 😊


VeADK 还支持你将短期记忆持久化存储在云端，未来的某一时刻你可以加载历史对话。

在使用云端记忆之前，需要安装`database`相关的依赖：

In [ ]:
!pip install veadk-python[database] --quiet

使用 MySQL 作为短期记忆的数据库后端：

In [ ]:
import os

os.environ["DATABASE_MYSQL_HOST"] = ""
os.environ["DATABASE_MYSQL_USER"] = ""
os.environ["DATABASE_MYSQL_PASSWORD"] = ""
os.environ["DATABASE_MYSQL_DATABASE"] = ""
os.environ["DATABASE_MYSQL_CHARSET"] = "utf8"

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk-playground-app"
user_id = "veadk-playground-user"
session_id = "veadk-playground-session"

agent = Agent()
short_term_memory = ShortTermMemory(backend="mysql") # 指定 mysql 后端
runner = Runner(agent=agent,
                short_term_memory=short_term_memory,
                app_name=app_name,
                user_id=user_id)

prompt = "我在 7 月 15 日购买了 20 个冰激凌"
response = await runner.run(messages=prompt, session_id=session_id)
print(response)

prompt = "我什么时候买了冰激凌？"
response = await runner.run(messages=prompt, session_id=session_id)
print(response)

原来你在7月15日买了20个冰激凌呀，数量不少呢！是有什么特别的场合需要用到这么多，还是单纯想囤货慢慢吃呀？ 😄 如果你需要记录这个购买信息、做简单的消费统计，或者有其他相关的小需求，随时告诉我，我可以帮你~
You mentioned that you bought the ice cream on July 15th. 😊


### 长期记忆

在使用记忆之前，需要安装`database`相关的依赖：



In [ ]:
!pip install veadk-python[database] --quiet

使用 OpenSearch 作为长期记忆的数据库后端：

In [ ]:
import os

# OpenSearch 配置
os.environ["DATABASE_OPENSEARCH_HOST"] = ""
os.environ["DATABASE_OPENSEARCH_PORT"] = ""
os.environ["DATABASE_OPENSEARCH_USERNAME"] = ""
os.environ["DATABASE_OPENSEARCH_PASSWORD"] = ""

# Embedding 配置（使用 OpenSearch 时，需要对文本进行向量化处理）
# 设置访问火山方舟的 Embedding 模型
os.environ["MODEL_EMBEDDING_NAME"] = "doubao-embedding-text-240715"
os.environ["MODEL_EMBEDDING_API_BASE"] = "https://ark.cn-beijing.volces.com/api/v3/embeddings"
os.environ["MODEL_EMBEDDING_DIM"] = "2560"
os.environ["MODEL_EMBEDDING_API_KEY"] = ""

In [ ]:
from veadk import Agent, Runner
from veadk.memory.long_term_memory import LongTermMemory
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk-playground-app"
user_id = "veadk-playground-user"

# 初始化一个长期记忆，采用 OpenSearch 向量化存储
# 长期记忆是跨 Session 的
long_term_memory = LongTermMemory(backend="opensearch")

agent = Agent(long_term_memory=long_term_memory)

runner = Runner(
    agent=agent,
    app_name=app_name,
    user_id=user_id,
    short_term_memory=ShortTermMemory(),
)


# ===== 插入记忆 =====
session_id = "veadk-playground-session"
teaching_prompt = "我上周五购买了一支冰激凌。"

await runner.run(messages=teaching_prompt, session_id=session_id)
await runner.save_session_to_long_term_memory(session_id=session_id) # 将 teaching prompt 和智能体回答保存到长期记忆中


# ===== 检验记忆 =====
session_id = "veadk-playground-session-2" # 使用一个新的 Session 来检测跨 Session 检索
student_prompt = "我上周五购买了什么? 用中文回答我。"

response = await runner.run(messages=student_prompt, session_id=session_id)

print(response)

你上周五购买了一支冰激凌。


### 知识库

由于知识库需要用到云端的向量化存储，因此在使用知识库之前，需要安装`database`相关的依赖：

In [ ]:
!pip install veadk-python[database] --quiet

使用 OpenSearch 作为知识库的后端：

In [ ]:
import os

# OpenSearch 配置
os.environ["DATABASE_OPENSEARCH_HOST"] = ""
os.environ["DATABASE_OPENSEARCH_PORT"] = "9200"
os.environ["DATABASE_OPENSEARCH_USERNAME"] = ""
os.environ["DATABASE_OPENSEARCH_PASSWORD"] = ""

# 设置访问火山方舟的 Embedding 模型
os.environ["MODEL_EMBEDDING_NAME"] = "doubao-embedding-text-240715"
os.environ["MODEL_EMBEDDING_API_BASE"] = "https://ark.cn-beijing.volces.com/api/v3/embeddings"
os.environ["MODEL_EMBEDDING_DIM"] = "2560"
os.environ["MODEL_EMBEDDING_API_KEY"] = ""

In [ ]:
from veadk import Agent, Runner
from veadk.knowledgebase.knowledgebase import KnowledgeBase
from veadk.memory.short_term_memory import ShortTermMemory

app_name = "veadk-playground-app"
user_id = "veadk-playground-user"
session_id = "veadk-playground-session"

knowledgebase_data = [
    "The secret of red is red_000000.",
    "The secret of green is green_000111.",
    "The secret of blue is blue_000222.",
    "The secret of yellow is yellow_000333.",
]

knowledgebase = KnowledgeBase(backend="opensearch") # 指定 opensearch 后端
knowledgebase.add(knowledgebase_data, app_name=app_name)

agent = Agent(knowledgebase=knowledgebase)

runner = Runner(
    agent=agent,
    short_term_memory=ShortTermMemory(),
    app_name=app_name,
    user_id=user_id,
)

response = await runner.run(messages="Tell me the secret of green.", session_id=session_id)
print(response)

The secret of green is green_000111.


## 调用追踪

### 本地观察

可以通过初始化 OpentelemetryTracer 来将 Agent 的调用栈存储为本地文件：

In [ ]:
import json

from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory
from veadk.tools.demo_tools import get_city_weather
from veadk.tracing.telemetry.opentelemetry_tracer import OpentelemetryTracer

app_name = "veadk-playground-app"
user_id = "veadk-playground-user"
session_id = "veadk-playground-session"

tracer = OpentelemetryTracer()

agent = Agent(tools=[get_city_weather], tracers=[tracer]) # 创建一个配置 tracers 的 Agent

short_term_memory = ShortTermMemory()

runner = Runner(
    agent=agent,
    short_term_memory=short_term_memory,
    app_name=app_name,
    user_id=user_id
    )

# 如果在 agent 中设置了tracer，runner将会自动尝试保存 tracing 文件至`/tmp`中
await runner.run(messages="北京的天气怎么样？", session_id=session_id)

print(f"Tracing file path: {tracer._trace_file_path}")

with open(tracer._trace_file_path, "r") as f:
    tracing_content = f.read()

print(f"Tracing file content:\n{json.loads(tracing_content)}")

Tracing file path: /tmp/veadk_global_tracer_veadk-playground-user_veadk-playground-session_b204302b2d0e24b6e26ca914a5fef478.json
Tracing file content:
[{'name': 'execute_tool get_city_weather', 'span_id': 179319007772593823, 'trace_id': 236624329413565051863190183785302783096, 'start_time': 1755172431614674977, 'end_time': 1755172431617289457, 'attributes': {'session.id': 'veadk-playground-session', 'user.id': 'veadk-playground-user', 'agent.name': 'veAgent', 'app.name': 'veadk-playground-app', 'gen_ai.system': 'gcp.vertex.agent', 'gen_ai.operation.name': 'execute_tool', 'gen_ai.tool.name': 'get_city_weather', 'gen_ai.tool.description': 'Retrieves the weather information of a given city. the args must in English', 'gen_ai.tool.call.id': 'call_2e7gig0mp86rzc1ffwf5ftdc', 'gcp.vertex.agent.tool_call_args': '{"city": "Beijing"}', 'gcp.vertex.agent.event_id': '04616e67-6881-43d0-8200-09c7c069be06', 'gcp.vertex.agent.tool_response': '{"result": "Sunny, 25°C"}', 'gcp.vertex.agent.llm_request'

### 云端观测

需要设置上报器（exporter），VeADK 当前支持将 Tracing 数据上报至火山引擎 TLS、火山引擎 APMPlus 以及火山引擎 Coze Loop 平台。它们分别对应不同的上报器。

初始化上报器之前需要进行环境变量设置：

In [ ]:
import os

# TLS 上报器环境变量
os.environ["OBSERVABILITY_OPENTELEMETRY_TLS_ENDPOINT"] = "https://tls-cn-beijing.volces.com:4318/v1/traces"
os.environ["OBSERVABILITY_OPENTELEMETRY_TLS_REGION"] = "cn-beijing"
os.environ["OBSERVABILITY_OPENTELEMETRY_TLS_SERVICE_NAME"] = ""

# APMPlus 上报器环境变量
os.environ["OBSERVABILITY_OPENTELEMETRY_APMPLUS_ENDPOINT"] = "http://apmplus-cn-beijing.volces.com:4317"
os.environ["OBSERVABILITY_OPENTELEMETRY_APMPLUS_SERVICE_NAME"] = ""
os.environ["OBSERVABILITY_OPENTELEMETRY_APMPLUS_API_KEY"] = ""

# Coze Loop 上报器环境变量
os.environ["OBSERVABILITY_OPENTELEMETRY_COZELOOP_ENDPOINT"] = "https://api.coze.cn/v1/loop/opentelemetry/v1/traces"
os.environ["OBSERVABILITY_OPENTELEMETRY_COZELOOP_SERVICE_NAME"] = ""
os.environ["OBSERVABILITY_OPENTELEMETRY_COZELOOP_API_KEY"] = ""

初始化上报器，并装配到 Agent 中：

In [ ]:
from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory
from veadk.tools.demo_tools import get_city_weather
from veadk.tracing.telemetry.exporters.apmplus_exporter import APMPlusExporter
from veadk.tracing.telemetry.exporters.cozeloop_exporter import CozeloopExporter
from veadk.tracing.telemetry.exporters.tls_exporter import TLSExporter
from veadk.tracing.telemetry.opentelemetry_tracer import OpentelemetryTracer

app_name = "veadk-playground-app"
user_id = "veadk-playground-user"
session_id = "veadk-playground-session"

exporters = [CozeloopExporter(), APMPlusExporter(), TLSExporter()] # 初始化 tracing 上报器
tracer = OpentelemetryTracer(exporters=exporters)

agent = Agent(tools=[get_city_weather], tracers=[tracer]) # 创建一个配置 tracers 的 Agent

runner = Runner(
    agent=agent,
    short_term_memory=short_term_memory,
    app_name=app_name,
    user_id=user_id
    )

await runner.run(messages="北京的天气怎么样？", session_id=session_id)

print(f"Tracing file path: {tracer._trace_file_path}")

Tracing file path: /tmp/veadk_global_tracer_veadk-playground-user_veadk-playground-session_65428d1063647316c8fa46579ba24121.json


最后，在云端平台可查看 Tracing 数据：


*   TLS 平台：https://www.volcengine.com/product/tls
*   APMPlus 平台：https://www.volcengine.com/product/apmplus
*   Coze Loop 平台：https://www.coze.cn/loop



## 评测

### 运行时数据转换

可以通过 runtime recorder 将运行时的数据直接保存为 Google ADK 兼容的 Evaluation Set 文件（json 格式）：

In [ ]:
import json

from veadk import Agent, Runner
from veadk.memory.short_term_memory import ShortTermMemory
from veadk.tools.demo_tools import get_city_weather

app_name = "veadk-playground-app"
user_id = "veadk-playground-user"
session_id = "veadk-playground-session"

agent = Agent(tools=[get_city_weather])
short_term_memory = ShortTermMemory()

runner = Runner(
    agent=agent,
    short_term_memory=short_term_memory,
    app_name=app_name,
    user_id=user_id
    )

await runner.run(messages="北京的天气怎么样？", session_id=session_id)

# 调用 runner 中的`save_eval_set`来保存评测集文件到本地
eval_set_path = await runner.save_eval_set(session_id=session_id)

print(f"Evaluation file path: {eval_set_path}")

with open(eval_set_path, "r") as f:
    data = json.load(f)
print(f"Evaluation file content\n{data}")

Evaluation file path: /tmp/veadk-playground-app/default.evalset.json
Evaluation file content
{'eval_set_id': 'default', 'name': 'default', 'description': None, 'eval_cases': [{'eval_id': 'veadk_eval_20250814115717', 'conversation': [{'invocation_id': 'e-55f9f9f1-0cd0-45d4-82e9-8d4805215e19', 'user_content': {'parts': [{'video_metadata': None, 'thought': None, 'inline_data': None, 'file_data': None, 'thought_signature': None, 'code_execution_result': None, 'executable_code': None, 'function_call': None, 'function_response': None, 'text': '北京的天气怎么样？'}], 'role': 'user'}, 'final_response': {'parts': [{'video_metadata': None, 'thought': None, 'inline_data': None, 'file_data': None, 'thought_signature': None, 'code_execution_result': None, 'executable_code': None, 'function_call': None, 'function_response': None, 'text': '北京当前天气晴朗，气温25°C。'}], 'role': None}, 'intermediate_data': {'tool_uses': [{'id': 'call_6wbw4k3urxyd4hy8cil83n9m', 'args': {'city': 'Beijing'}, 'name': 'get_city_weather'}], '

### 使用评测器进行评测

在使用评测之前，需要安装评测相关的依赖：

In [ ]:
!pip install veadk-python[eval] --quiet

评判模型的环境变量设置：

In [ ]:
import os

os.environ["MODEL_JUDGE_API_BASE"] = "https://ark.cn-beijing.volces.com/api/v3/"
os.environ["MODEL_JUDGE_NAME"] = "doubao-seed-1-6-250615"
os.environ["MODEL_JUDGE_API_KEY"] = ""

以 Deepval 为例：

In [ ]:
from deepeval.metrics import GEval, ToolCorrectnessMetric
from deepeval.test_case import LLMTestCaseParams

from veadk import Agent
from veadk.tools.demo_tools import get_city_weather
from veadk.config import getenv
from veadk.evaluation.deepeval_evaluator import DeepevalEvaluator
from veadk.prompts.prompt_evaluator import eval_principle_prompt

agent = Agent(tools=[get_city_weather])

evaluator = DeepevalEvaluator(agent=agent,
                              judge_model_api_key=getenv("MODEL_JUDGE_API_KEY"))

judge_model = evaluator.judge_model # 如果你希望使用方舟作为 judge model，那么 evaluator 会帮你自动初始化

metrics = [
    GEval(
        threshold=0.8,
        name="Base Evaluation",
        criteria=eval_principle_prompt,
        evaluation_params=[
            LLMTestCaseParams.INPUT,
            LLMTestCaseParams.ACTUAL_OUTPUT,
            LLMTestCaseParams.EXPECTED_OUTPUT,
        ],
        model=judge_model, # 这里需要传入 judge model
    ),
    ToolCorrectnessMetric(
        threshold=0.5
    ),
]

await evaluator.eval(eval_set_file_path=eval_set_path, metrics=metrics)

✨ You're running DeepEval's latest Base Evaluation [GEval] Metric! (using None (Local Model), strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Tool Correctness Metric! (using None, strict=False, async_mode=True)...

Output()



Metrics Summary

  - ✅ Base Evaluation [GEval] (score: 1.0, threshold: 0.8, strict: False, evaluation model: None (Local Model), reason: All evaluation steps are satisfied. The Actual Output ('北京当前天气晴朗，气温为25°C。'), Input ('北京的天气怎么样？'), and Expected Output ('北京当前天气晴朗，气温25°C。') are all human-readable (coherent, no garbled text). Both Actual and Expected Outputs are relevant to the Input, which asks about Beijing's weather., error: None)
  - ✅ Tool Correctness (score: 1.0, threshold: 0.5, strict: False, evaluation model: None, reason: All expected tools ['get_city_weather'] were called (order not considered)., error: None)

For test case:

  - input: 北京的天气怎么样？
  - actual output: 北京当前天气晴朗，气温为25°C。
  - expected output: 北京当前天气晴朗，气温25°C。
  - context: ['actual_conversation_history: Empty', 'expect_conversation_history: Empty']
  - retrieval context: None


Overall Metric Pass Rates

Base Evaluation [GEval]: 100.00% pass rate
Tool Correctness: 100.00% pass rate




✓ Evaluation completed 🎉! (time taken: 24.09s | token cost: 0.0 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» What to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Base Evaluation [GEval]', threshold=0.8, success=True, score=1.0, reason="All evaluation steps are satisfied. The Actual Output ('北京当前天气晴朗，气温为25°C。'), Input ('北京的天气怎么样？'), and Expected Output ('北京当前天气晴朗，气温25°C。') are all human-readable (coherent, no garbled text). Both Actual and Expected Outputs are relevant to the Input, which asks about Beijing's weather.", strict_mode=False, evaluation_model='None (Local Model)', error=None, evaluation_cost=0.0, verbose_logs='Criteria:\n\nYou are a LLM for evaluating other models\' responses. Note:\n- The response maybe generated by some uncertainty tools (e.g., online-search, random number), you just need to consider whether the response is human-readable, rather than focus on the specific content. Because the specific content maybe different at different time.\n \n \nEvaluation Steps:\n[\n    "Check if the Actual Output is human-readable (co

## Prompt 优化

使用火山引擎 PromptPilot 来进行 Agent 的系统提示词（System Prompt）优化。

In [ ]:
# 安装火山引擎提供的依赖
!pip install agent-pilot-sdk>=0.0.9

设置 KEY 来访问服务：

In [ ]:
import os
os.environ["AGENT_PILOT_API_KEY"] = ""

定义一个简单的 Agent，并且让它携带天气查询的工具，体验 Prompt 优化前后的差异。

In [ ]:
from veadk import Agent
from veadk.tools.demo_tools import get_city_weather

agent = Agent(instruction="你是一个智能体", tools=[get_city_weather])

使用 Prompt Pilot 进行优化：

In [ ]:
from veadk.cli.services.agentpilot.agentpilot import AgentPilot

agent_pilot = AgentPilot(api_key=os.getenv("AGENT_PILOT_API_KEY"))

refined_prompt = agent_pilot.optimize(agents=[agent])

print(refined_prompt)

Optimized prompt for agent veAgent:
# Role
你是veAgent，是由VeADK团队开发的AI智能体，擅长数据科学、文档处理和软件开发领域。

# Task Requirements
## 对话风格
- 语言表达应简洁明了，避免冗长复杂的表述，确保信息高效传递。
- 态度友好亲切，展现出专业且耐心的服务态度。

## 沟通目的
- 根据用户的需求，提供准确、专业的解决方案和建议。
- 若用户咨询数据科学、文档处理或软件开发相关问题，应基于专业知识进行解答。
- 若用户需要获取特定城市的天气信息，可调用get_city_weather工具查询并告知。

## 其他要求
- 回复内容需紧密围绕用户的问题，避免偏离主题。
- 若对用户的问题理解存在疑问，应及时询问以明确需求。 
# Role
你是veAgent，是由VeADK团队开发的AI智能体，擅长数据科学、文档处理和软件开发领域。

# Task Requirements
## 对话风格
- 语言表达应简洁明了，避免冗长复杂的表述，确保信息高效传递。
- 态度友好亲切，展现出专业且耐心的服务态度。

## 沟通目的
- 根据用户的需求，提供准确、专业的解决方案和建议。
- 若用户咨询数据科学、文档处理或软件开发相关问题，应基于专业知识进行解答。
- 若用户需要获取特定城市的天气信息，可调用get_city_weather工具查询并告知。

## 其他要求
- 回复内容需紧密围绕用户的问题，避免偏离主题。
- 若对用户的问题理解存在疑问，应及时询问以明确需求。 
